In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from IPython.display import display
from datetime import datetime, timedelta
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pickle
import os

scopes = ['https://www.googleapis.com/auth/calendar']

In [3]:
with open('footy_teams.txt', 'r') as ofile:
    team_content = ofile.readline()
    ## reading the content from the file

team_content = team_content.split(',')
team_content = [x.strip() for x in team_content]
team_content = team_content[:-1]
## splitting every team content into the list format

month_name = {'January': 1, 
       'February': 2,
       'March': 3,
       'April': 4,
       'May': 5,
       'June': 6,
       'July': 7,
       'August': 8,
       'September': 9,
       'October': 10,
       'November': 11, 
       'December': 12
      }

print(team_content)

['Barcelona']


In [88]:
for team in team_content:
    search_term = f'{team} sky sports fixtures'
    
    print('\nGetting The Link of the website...\n')
    
    ## accessing the link of the website
    page = requests.get(f"https://www.google.dz/search?q={search_term}")
    soup = BeautifulSoup(page.content)
    links = soup.findAll("a")
    for link in  soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
        temp_link = re.split(":(?=http)",link["href"].replace("/url?q=",""))
        break

    website_link = temp_link[0].split('&')[0]

    print(f'\nScrapping data for {team} from the website...\n')
    print(website_link)
    
    ## scraping the content from the website
    scrape_data = requests.get(website_link)
    soup = BeautifulSoup(scrape_data.text, 'html.parser')
    
    ## finding the div tag which contains all fixture's information
    results = soup.find('div', attrs={'class': 'fixres__body'})
    
    ## scrapping fixuture's date, competition name, team names and timing of the fixtures
    years = results.find_all('h3')
    fix_date = results.find_all('h4')
    comp_name = results.find_all('h5')
    teams = results.find_all('span', attrs={'class': 'swap-text__target'})
    timings = results.find_all('span', attrs={'class': 'matches__date'})
    
    ## making a dict of all years
    year_dict = {}
    
    for year in years:
        year_temp = year.text.split(' ')
        year_dict[year_temp[0]] = year_temp[1]
    
    ## making a list of all dates
    date_text = []

    for date in fix_date:
        temp = date.text.split(' ')
        
        date_final =''
        
        for i in temp[1]:
            if i.isdigit():
                date_final += i
            else:
                break
                
        year = year_dict[temp[2]]
        month = month_name[temp[2]]
        
        date_text.append(f'{date_final} {str(month)} {year}')
    
    ## making a list of all competition names
    comp_text = []

    for comp in comp_name:
        comp_text.append(comp.text)
    
    ## making a list of match times
    match_time = []

    for time in timings:
        match_time.append(time.text.strip())
        
    final_time = []
        
    for date_x, time_y in zip(date_text, match_time):
        date_split = date_x.split(' ')
        date_split = [int(elem) for elem in date_split]
        time_split = time_y.split(':')
        time_split = [int(elem) for elem in time_split]
        temp_time = datetime(date_split[2], date_split[1], date_split[0], time_split[0], time_split[1], 0)
        temp_time = temp_time + timedelta(hours=4)
        final_time.append(temp_time)
    
    ## making a dictionary that will contain all information

    final_record = dict()
    home_team = []
    away_team = []
    
    count = 1
    
    for x_temp in teams:
        if x_temp.text == '\n\n\n\n':
            continue
        
        elif count % 2 != 0:
            home_team.append(x_temp.text)
        else:
            away_team.append(x_temp.text)
        count += 1

    for i in range(len(match_time)):
        final_record['Date/Time'] = final_time
        final_record['Competition'] = comp_text
    
    final_record['Home_Team'] = home_team
    final_record['Away_Team'] = away_team
    
    print('\nScrapped Successfully')
    display(pd.DataFrame(final_record))


Getting The Link of the website...


Scrapping data for Barcelona from the website...

https://www.skysports.com/barcelona-fixtures

Scrapped Successfully


,Date/Time,Competition,Home_Team,Away_Team
0,2020-03-07 21:30:00,Spanish La Liga,Barcelona,Real Sociedad
1,2020-03-14 21:30:00,Spanish La Liga,Real Mallorca,Barcelona
2,2020-03-19 00:00:00,UEFA Champions League,Barcelona,Napoli
3,2020-03-23 00:00:00,Spanish La Liga,Barcelona,Leganes
4,2020-04-05 21:30:00,Spanish La Liga,Sevilla,Barcelona
5,2020-04-12 21:30:00,Spanish La Liga,Barcelona,Athletic Bilbao
6,2020-04-23 00:00:00,Spanish La Liga,Celta Vigo,Barcelona
7,2020-04-26 21:30:00,Spanish La Liga,Barcelona,Atletico Madrid
8,2020-05-03 21:30:00,Spanish La Liga,Villarreal,Barcelona
9,2020-05-10 21:30:00,Spanish La Liga,Barcelona,Espanyol


In [212]:
if 'token.pkl' in os.listdir():
    with open('token.pkl', 'rb') as f:
        credentials = pickle.load(f)
else:
    flow = InstalledAppFlow.from_client_secrets_file('client_secret.json', scopes=scopes)
    credentials = flow.run_console()
    pickle.dump(credentials, open('token.pkl', 'wb'))

service = build('calendar', 'v3', credentials=credentials)

result = service.calendarList().list().execute()
calendar_id = result['items'][0]['id']
result = service.events().list(calendarId=calendar_id).execute()

In [213]:
result

{'kind': 'calendar#events',
 'etag': '"p3309jh6rti1ug0g"',
 'summary': 'anmoldurgapal612@gmail.com',
 'updated': '2020-03-05T17:23:26.200Z',
 'timeZone': 'Asia/Kolkata',
 'accessRole': 'owner',
 'defaultReminders': [{'method': 'popup', 'minutes': 30}],
 'nextPageToken': 'CigKGjd2NG9vaWF0ZGMxZDhnbXFnZHR1NmZzYWwwGAEggICAsM7sroUXGg0IABIAGMCZxNvsg-gCIgcIAhCgqdUC',
 'items': []}

In [90]:
timezone = 'Asia/Kolkata'

In [92]:
def even_struct(summary, desc, start_time, timezone):
    end_time = start_time + timedelta(hours=3)
    
    event = {
      'summary': summary,
      'description': desc,
      'start': {
        'dateTime': start_time.strftime("%Y-%m-%dT%H:%M:%S"),
        'timeZone': timezone,
      },
      'end': {
        'dateTime': end_time.strftime("%Y-%m-%dT%H:%M:%S"),
        'timeZone': timezone,
      },
      'reminders': {
        'useDefault': False,
        'overrides': [
          {'method': 'popup', 'minutes': 30},
        ],
      },
    }
    
    return event

In [210]:
result = service.events().list(calendarId=calendar_id).execute()
summary = []
desc = []
start_time = []
result_dict = {}

for item in range(len(result['items'])):
    temp = result['items'][item]['summary'].split(' ')[-1]
    if temp == '(Football)':
        summary.append(result['items'][item]['summary'])
        desc.append(result['items'][item]['description'])
        date_temp = result['items'][item]['start']['dateTime']
        date_temp = date_temp.split('-')
        day = date_temp[2][:2]
        day = [elem for elem in day if elem != '0']
        day = ''.join(elem for elem in day)
        month = date_temp[1]
        month = [elem for elem in month if elem != '0']
        month = ''.join(elem for elem in month)
        year = date_temp[0]
        stime = f'{year} {month} {day}'
        start_time.append(stime)

result_dict['Summary'] = summary
result_dict['Description'] = desc
result_dict['start_time'] = start_time

In [211]:
result_dict

{'Summary': [], 'Description': [], 'start_time': []}

In [209]:
for i in range(len(final_record['Home_Team'])):
    temp_list = []
    
    for index in final_record:
        temp_list.append(final_record[index][i])
        
    start_time = temp_list[0]
    desc = temp_list[1]
    summary = f'{temp_list[2]} vs {temp_list[3]} (Football)'
    day = start_time.strftime('%d')
    day = ''.join(elem for elem in day if elem != '0')
    month = start_time.strftime('%m')
    month = ''.join(elem for elem in month if elem != '0')        
    year = start_time.strftime('%Y')
    start_time_temp = f'{year} {month} {day}'
    count = False
    for j in range(len(result_dict['Summary'])):
        if summary == result_dict['Summary'][j] and desc == result_dict['Description'][j]:
            if start_time_temp != result_dict['start_time'][j]:
                print('Update Event Code')
                count = True
                break
            else:
                count = True
                break
    if count == True:
        print('No Operation')
    else:
        print(f'Adding Event {summary}')
        event = even_struct(summary, desc, start_time, timezone)
        service.events().insert(calendarId=calendar_id, body=event).execute()

Adding Event Barcelona vs Real Sociedad (Football)
Adding Event Real Mallorca vs Barcelona (Football)
Adding Event Barcelona vs Napoli (Football)
Adding Event Barcelona vs Leganes (Football)
Adding Event Sevilla vs Barcelona (Football)
Adding Event Barcelona vs Athletic Bilbao (Football)
Adding Event Celta Vigo vs Barcelona (Football)
Adding Event Barcelona vs Atletico Madrid (Football)
Adding Event Villarreal vs Barcelona (Football)
Adding Event Barcelona vs Espanyol (Football)
Adding Event Real Valladolid vs Barcelona (Football)
Adding Event Barcelona vs Osasuna (Football)
Adding Event Alaves vs Barcelona (Football)
